# smallest-free-number
A coding challenge to find the smallest non-negative integer that is not part of a given set of non-negative integers.

Thanks to [Nicolas Rinaudo](https://github.com/abcoates/smallest-free-number.git) for suggesting this problem for a coding challenge.

## Description
This is a simplification a general problem, find the 'least XXX' object that is not already used, where 'XXX' is some arbitrary measurement dimension such as 'large', 'expensive', etc.

In this minimal version, you are given a set (i.e. an **unordered** set) of non-negative integers, and you have to find the smallest non-negative integer that is **not** a part of the set.

## Examples
 * \[0, 1, 2, 3, 5\] => 4
 * \[0, 1, 3, 4, 5\] => 2
 * \[2, 1, 0\] => 3
 * \[20, 10, 30\] => 0

## Special Note
You **may** use AI to help you write the code.  As AI coding companions are inevitable, we might as well all start practicising how to use them.  **However**, if you have used AI to help you write your code, please make that clear in your solution.

## Stretch Goal
Finding a solution is straightforward, but can you find a solution that only takes linear time?  Measure how the time taken for your solution varies as the size of the set is increased, and see how close you can get to the time taken growing linearly with the size of the set.  Create a graph of the time measurements again set size.

You will need to generate unordered sets of numbers that are sufficiently large to make the solution time sufficiently measurable.

Your results may vary with the **density** of the sets, i.e. with the percentage of unused numbers in the range from zero to the largest number in the set.

## Solution - F#

### Solution #1 - Brute Force

As a first solution, let's try something "brute force" - sort the set, then walk up the integers from zero until we find one that isn't in the set.

In [ ]:
let solution1 (intset: int list): int = // 'intset' must be a set in unordered list format - to avoid pre-sorted F# sets giving an advantage
    let sortedset = intset |> List.sort
    let rec find (sortedlist: int list) (nextValue: int) =
        match sortedlist with
        | [] -> nextValue
        | other ->
            let listhead = sortedlist |> List.head
            if (nextValue < listhead)
            then nextValue
            elif (nextValue = listhead)
            then find (sortedlist |> List.tail) (nextValue+1)
            else find(sortedlist |> List.tail) nextValue
    find sortedset 0

\[0, 1, 2, 3, 5\] => 4

In [ ]:
solution1 [0;1;2;3;5]

\[0, 1, 3, 4, 5\] => 2

In [ ]:
solution1 [0;1;3;4;5]

\[2, 1, 0\] => 3

In [ ]:
solution1 [2;1;0]

\[20, 10, 30\] => 0

In [ ]:
solution1 [20;10;30]

So 'solution1' works correctly.  However, how performant is it?

We'll need a graphing package - 'XPlot' will do nicely.

In [ ]:
#r "nuget: XPlot.Plotly"
#r "nuget: XPlot.Plotly.Interactive"

open XPlot.Plotly

Let's also get a package for curve fitting.

In [ ]:
#r "nuget: MathNet.Numerics"

open MathNet.Numerics

We'll need functions that can generate random sets of non-negative integers.

**However**, F# seems to implement sets as sorted lists, which makes it too easy to find the smallest unused integer.  As such, I will need to return the set as a list in which the order of the elements has been randomised.

**Update:** instead, I'll just generate the worst-case, highest-to-lowest set of integers.

In [ ]:
// open System

// let randomGenerator = Random()

In [ ]:
// let generateSet (maximum:int) (length:int): int list =
//     let rec iterate (currentList: int list) (currentSet: int Set): int list =
//         match currentSet with
//         | finished when (finished |> Set.count) >= length -> currentList
//         | unfinished ->
//             let newVal = randomGenerator.Next(0, maximum+1)
//             if (currentSet |> Set.contains newVal)
//             then
//                 iterate currentList currentSet
//             else
//                 iterate (newVal::currentList) (currentSet |> Set.add newVal)
//     iterate [] ([] |> Set.ofList)

In [ ]:
let generateSet (length:int): int list = seq {length..(-1)..1} |> Seq.toList

In [ ]:
let set1 = generateSet 5
set1

In [ ]:
solution1 set1

We will do many runs where the timing is an integer number of microseconds, so we will need a quick way to calculate the integer average time of those runs.

**Update:** no, we'll only do a single worst-case run.

### Time Solution #1

The following function can be used to time how long a solution takes to run.  It averages the time over ten runs.

In [ ]:
let timeSolution (length: int) (solution: int list -> int): int*int =
    let timeOnce (): int*int*int =
        GC.Collect()
        let testSet = generateSet length
        let sw = System.Diagnostics.Stopwatch.StartNew()
        let result = solution testSet
        sw.Stop()
        (length, result, sw.Elapsed.Microseconds)
    for counter in 1..20 do // warm-up
        timeOnce () |> ignore
    let timings = seq {
        for counter in 1..10 do
            let (_, _, timing) = timeOnce ()
            yield timing
    }
    (length, ((timings |> Seq.sum)+5)/10)

In [ ]:
timeSolution 100 solution1

In [ ]:
timeSolution 100 solution1

In [ ]:
timeSolution 200 solution1

In [ ]:
timeSolution 200 solution1

Let's create timings (in microseconds) for set sizes up to 5 000.  To make it as hard as possible, we'll set 'maximum = length', meaning that there is only one number in the range '0..maximum' which does not appear in the set.

In [ ]:
let maxLength = 30000
let graphSegments = 600
let increment = maxLength / graphSegments
let lengths = {increment..increment..maxLength} |> Seq.toList
let maximums = lengths
let pairs: (int*int) list = List.zip maximums lengths
let timings = pairs |> List.map (fun pair ->
    let (maximum, length) = pair
    printfn "Timing length %d ..." length
    let (_, timing) = timeSolution length solution1
    timing
)
printfn "Done."

In [ ]:
List.zip lengths timings

Let's see what the total elapsed calculation time in seconds is.

In [ ]:
let totalElapsedCalculationTimeInMicrosec = (timings |> List.sum) * 20 |> float
let totalElapsedCalculationTimeInSeconds = totalElapsedCalculationTimeInMicrosec / 1000000.
totalElapsedCalculationTimeInSeconds

Now let's try fitting an O(n^q) curve to the results.  I use a semi-calculated, semi-empirical starting point, with a starting order of O(n^1.5).

In [ ]:
let lengthsFloat = lengths |> List.map float |> Array.ofList
let timingsFloat = timings |> List.map float |> Array.ofList

let first<'a> (arr: 'a[]) = arr[0]
let last<'a> (arr: 'a[]) = arr[arr.Length - 1]

// These scaling factors are used to give the parameters p0, p1 and p2 similar magnitudes, which tends to work best with fitting algorithms.
let p2scale = 0.75 // assuming O(1.5) as a starting point
let p1scale = ((last timingsFloat) - (first timingsFloat)) / ((last lengthsFloat) - (first lengthsFloat))
let p0scale = (first timingsFloat) - p1scale * (first lengthsFloat)
(p0scale, p1scale, p2scale)

In [ ]:

let fitFunc (p0:float) (p1:float) (p2:float) (x:float) =
    // printfn "p0 = %f, p1 = %f, p2 = %f, x = %f" p0 p1 p2 x
    (p0*p0scale) + (p1*p1scale) * x**(p2*p2scale)

let p0init = 1.
let p1init = 1.
let p2init = 2. // assuming O(1.5) as a starting point
let tolerance = 0.1
let maxIterations = 10000
(p0init, p1init, p2init)

In [ ]:
let (p0, p1, p2) = Fit.Curve(lengthsFloat, timingsFloat, fitFunc, p0init, p1init, p2init, tolerance, maxIterations).ToTuple()
(p0, p1, p2)

OK, let's plot the measurements and the fit together.

In [ ]:
let measurementCurve = Scatter(x = lengths, y = timings, name="Measurements")

let fitLengths = {(lengths |> List.min)..(lengths |> List.max)} |> Seq.map float |> Array.ofSeq
let fitTimings = fitLengths |> Array.map (fun x -> fitFunc p0 p1 p2 x)
let fitCurve = Scatter(x = fitLengths, y = fitTimings, name="O(n) Fit Curve")

[measurementCurve; fitCurve]
|> Chart.Plot
|> Chart.WithTitle("Solution Time vs. Set Size")
|> Chart.WithXTitle("Set Size")
|> Chart.WithYTitle("Solution Time in Microsec")

In [ ]:
let q = p2*p2scale
q

So, approximately, the order of 'solution1' is close to O(n) - not at all what was expected of a "brute force" approach.

## Discussion #1

Why is the order so close to 1, for what was intended to be a 'brute force' solution?

It may be that list sorting is *so* optimised in F#/.NET that we don't see the cost of it, so you only see the cost of walking up to find the first unused integer, which is naturally O(n).

## Solution #2 - More Functional - no use of unique sets

In [ ]:
let solution2 (intset: int list): int =
    let rec find (intset: int list) (next: int): int =
        match intset with
        | [] -> next
        | head::tail when head = next -> find tail (next+1)
        | other ->
            let remainderLow, remainderHigh = intset |> List.partition (fun n -> n <= next)
            if (remainderLow |> List.isEmpty)
            then next
            else find remainderHigh (next+1)
    find intset 0

\[0, 1, 2, 3, 5\] => 4

In [ ]:
solution2 [0;1;2;3;5]

\[0, 1, 3, 4, 5\] => 2

In [ ]:
solution2 [0;1;3;4;5]

\[2, 1, 0\] => 3

In [ ]:
solution2 [2;1;0]

\[20, 10, 30\] => 0

In [ ]:
solution2 [20;10;30]

### Time Solution #2

In [ ]:
timeSolution 100 solution2

In [ ]:
timeSolution 100 solution1

In [ ]:
timeSolution 200 solution1

In [ ]:
timeSolution 200 solution1

In [ ]:
let maxLength = 30000
let graphSegments = 600
let increment = maxLength / graphSegments
let lengths = {increment..increment..maxLength} |> Seq.toList
let maximums = lengths
let pairs: (int*int) list = List.zip maximums lengths
let timings = pairs |> List.map (fun pair ->
    let (maximum, length) = pair
    printfn "Timing length %d ..." length
    let (_, timing) = timeSolution length solution2
    timing
)
printfn "Done."

In [ ]:
List.zip lengths timings

Let's see what the total elapsed calculation time in seconds is.

In [ ]:
let totalElapsedCalculationTimeInMicrosec = (timings |> List.sum) * 20 |> float
let totalElapsedCalculationTimeInSeconds = totalElapsedCalculationTimeInMicrosec / 1000000.
totalElapsedCalculationTimeInSeconds

Now let's try fitting an O(n^q) curve to the results.  I use a semi-calculated, semi-empirical starting point, with a starting order of O(n^1.5).

In [ ]:
let lengthsFloat = lengths |> List.map float |> Array.ofList
let timingsFloat = timings |> List.map float |> Array.ofList

let first<'a> (arr: 'a[]) = arr[0]
let last<'a> (arr: 'a[]) = arr[arr.Length - 1]

// These scaling factors are used to give the parameters p0, p1 and p2 similar magnitudes, which tends to work best with fitting algorithms.
let p2scale = 0.75 // assuming O(1.5) as a starting point
let p1scale = ((last timingsFloat) - (first timingsFloat)) / ((last lengthsFloat) - (first lengthsFloat))
let p0scale = (first timingsFloat) - p1scale * (first lengthsFloat)
(p0scale, p1scale, p2scale)

In [ ]:

let fitFunc (p0:float) (p1:float) (p2:float) (x:float) =
    // printfn "p0 = %f, p1 = %f, p2 = %f, x = %f" p0 p1 p2 x
    (p0*p0scale) + (p1*p1scale) * x**(p2*p2scale)

let p0init = 1.
let p1init = 1.
let p2init = 2. // assuming O(1.5) as a starting point
let tolerance = 0.1
let maxIterations = 10000
(p0init, p1init, p2init)

In [ ]:
let (p0, p1, p2) = Fit.Curve(lengthsFloat, timingsFloat, fitFunc, p0init, p1init, p2init, tolerance, maxIterations).ToTuple()
(p0, p1, p2)

OK, let's plot the measurements and the fit together.

In [ ]:
let measurementCurve = Scatter(x = lengths, y = timings, name="Measurements")

let fitLengths = {(lengths |> List.min)..(lengths |> List.max)} |> Seq.map float |> Array.ofSeq
let fitTimings = fitLengths |> Array.map (fun x -> fitFunc p0 p1 p2 x)
let fitCurve = Scatter(x = fitLengths, y = fitTimings, name="O(n) Fit Curve")

[measurementCurve; fitCurve]
|> Chart.Plot
|> Chart.WithTitle("Solution Time vs. Set Size")
|> Chart.WithXTitle("Set Size")
|> Chart.WithYTitle("Solution Time in Microsec")

In [ ]:
let q = p2*p2scale
q

So, approximately, the order of 'solution1' is close to O(n).

## Appendix

**Note from Nicolas** If your set length is N, then your solution value must be in the range 0..N.